In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
transformer = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# transformer.save('./model')
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
texts = [
    "Титан 2 - это дружная команда, которая работат лучше всех!",
    "Команда сигнал победила в городскай эстафете так как это дружная команда!",
    "Титан 2 - это слаженная команда, сродни семье!"
    ]

embeddings = transformer.encode(texts)

cos_sim_matrix = np.zeros((3, 3))

for i in range(3):
    line = []
    for j in range(3):
        line.append(util.cos_sim(embeddings[i], embeddings[j]).item())

    cos_sim_matrix[i] += line

sns.heatmap(cos_sim_matrix, annot=True)
plt.title("Сходство предложений")


NameError: name 'transformer' is not defined

In [23]:
import json 

data = {}

with open("C:\\Работа\\FAQ Титан 2 - Ваня Сухоруков\\data\\test_data.json", "r", encoding="utf8") as f:
    data = json.load(f)

data

[{'question': 'Какой объект выступает в качестве объекта прикладной экологии в самом широком смысле?',
  'answer': 'Объектом прикладной экологии в самом широком смысле является совокупность всех частных объектов экологических изысканий, взаимосвязанных в рамках геоэкологического пространства.'},
 {'question': 'Какие элементы присутствуют в определении объекта прикладной экологии?',
  'answer': 'В определении объекта прикладной экологии присутствуют следующие элементы: 1) материальные тела, 2) указание на их взаимосвязь, 3) территориальная привязка в рамках геоэкологического пространства.'},
 {'question': 'Что подразумевается под материальными телами в контексте экологических изысканий?',
  'answer': 'Под материальными телами подразумеваются частные объекты экологических изысканий.'},
 {'question': 'Какие системы относятся к объектам экологических изысканий?',
  'answer': 'К объектам экологических изысканий относятся различные антропогенные, естественно-антропогенные и естественные сист

In [27]:
[data[i]['question'] for i in range(5)]

['Какой объект выступает в качестве объекта прикладной экологии в самом широком смысле?',
 'Какие элементы присутствуют в определении объекта прикладной экологии?',
 'Что подразумевается под материальными телами в контексте экологических изысканий?',
 'Какие системы относятся к объектам экологических изысканий?',
 'Какие роли выполняет человек в экологических отношениях?']

In [4]:
import os
import pickle

class TextBase:
    def __init__(self, db_path: str = "C:\\Работа\\FAQ Титан 2 - Ваня Сухоруков\\base.pkl"):
        self.db_path = db_path
        self.base = self._load_db() if os.path.exists(db_path) else {"Question": [], "Answer": []}

        self.encoder = Encoder()

    def add_qna(self, qna_list: list[dict[str, str]]):
        """qna_list = [{'question': '...', 'answer': '...'}, {'question': '...', 'answer': '...'}, ...]"""
        questions = [elem['question'] for elem in qna_list]
        answeres = [elem['answer'] for elem in qna_list]

        questions_embeddings = self.encoder.encode(questions)
        # answeres_embeddings = self.encoder.encode(answeres)

        self.base['Question'].extend(questions_embeddings)
        self.base['Answer'].extend(answeres)

        self._save()



    def find_answer(self, question):
        question_embedding = self.encoder.encode(question)

        cos_sim = util.cos_sim(question_embedding, self.base['Question'])

        if cos_sim.max().item() < 0.6:
            return "Не знаю что ответить"
        
        return self.base["Answer"][np.argmax(cos_sim)]
        

    def _load_db(self):
        with open(self.db_path, 'rb') as f:
            return pickle.load(f)
        
    def _save(self):
        with open(self.db_path, 'wb') as f:
            pickle.dump(self.base, f)
        
class Encoder:
    def __init__(self, model_path='C:\\Работа\\FAQ Титан 2 - Ваня Сухоруков\\model'):
        self.model = SentenceTransformer.load(model_path)

    def encode(self, text):
        return self.model.encode(text)
    
db = TextBase()
# db.add_qna(data)



    


In [8]:
test = [
    "Выдающиеся учёные экологи?",
    "Какой вклад внёс Чарльз Дарвин в развитие экологии?",
    "Экология в 20 веке?",
    "Экология в 19 веке?"
]


for text in test:
    res = db.find_answer(text)
    print(f'Q: {text}\nA: {res}')

Q: Выдающиеся учёные экологи?
A: Ю. Андерсен, Э. Пианка, Р. Риклефс, Ф. Швейцер, Дж. Харпер, Р. Уиттекер, Н. Борлауг, Т. Миллер, Б. Небел и др.
Q: Какой вклад внёс Чарльз Дарвин в развитие экологии?
A: Этот труд считается первым трудом по экологии, где сформулирован вывод о борьбе за существование.
Q: Экология в 20 веке?
A: В конце XX века происходит «экологизация» науки, связанная с осознанием огромной роли экологических знаний.
Q: Экология в 19 веке?
A: Во второй половине XIX века содержанием экологии было главным образом изучение образа жизни животных и растений, их адаптивности к климатическим условиям.
